# **Monitor de Concorrentes com LLMs e Gradio**

No mundo dos negócios, **monitorar a concorrência** é essencial para se manter competitivo. Saber o que os concorrentes estão fazendo — como promoções, lançamentos de novos produtos, mudanças de preços e feedbacks dos clientes — pode ajudar uma empresa a ajustar suas estratégias e tomar decisões mais informadas. No entanto, fazer isso manualmente é demorado e ineficiente.

É aí que entra o **Monitor de Concorrentes com LLMs e Gradio**: um sistema automatizado que coleta, analisa e resume informações relevantes de sites de concorrentes, fornecendo insights valiosos de forma rápida e eficiente.



## Importando as bibliotecas

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import pandas as pd


In [2]:
# Carrega as variáveis de ambiente a partir do arquivo .env
# O parâmetro 'override=True' permite sobrescrever variáveis já carregadas
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    
    print(f"A chave de API OpenAI existe e começa {openai_api_key[:8]}")
else:
    
    print("Chave de API OpenAI não definida")


A chave de API OpenAI existe e começa sk-proj-


In [3]:
# Conectando a API da OpenAI, 

openai = OpenAI()

In [4]:
# Uma classe para representar uma página da Web

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Nenhum título encontrado"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
# Função para enviar o conteúdo para o LLM e gerar o resumo em Markdown
def stream_gpt(prompt):
    system_message = """
    Você é um assistente especializado em análise de concorrência. Analise páginas da web e destaque: promoções, descontos e ofertas;
    novos produtos ou serviços;
    Seja conciso, direto e utilize Markdown para organizar os insights.
    """
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    # Criação da requisição para o modelo GPT-4
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',  # Ou o modelo que preferir
        messages=messages,
        stream=True
    )
    
    result = ""
    # Processa o streaming, gerando partes da resposta
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result  # Emite cada parte da resposta conforme gerada


In [6]:
# Função principal que integra a coleta de dados da página com o LLM
def monitorar_concorrente(url):
    website = Website(url)
    contents = website.get_contents()
    
    # Geração do resumo em Markdown com o LLM utilizando o gerador
    result = stream_gpt(contents)
    
    # Usando 'yield from' para emitir as partes do gerador
    yield from result

# Interface Gradio
iface = gr.Interface(fn=monitorar_concorrente,
                     inputs=gr.Textbox(label="URL do Concorrente"),
                     outputs=gr.Markdown(label="Resumo do Concorrente"),
                     title="Monitor de Concorrentes com LLM",
                     description="Insira a URL de um concorrente para coletar e analisar informações como promoções, novos produtos e muito mais. O modelo gerará um resumo com insights em formato Markdown.")
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
# gerando um link público
iface.launch(share=True)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* Running on public URL: https://3ade0b576cec4eb787.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
